# Quanta Maths: Integer Addition and Subtraction in Transformers. Scan 200+ LLMs

This Colab uses the app.withmartian.com API to test 200+ models
to see whether given query like "Answer concisely: 4444+5559=" they answer "10003".

In [1]:
# Non-obvious cascading carry tests
# Units column ≥10, all other columns sum to exactly 9
add_tests = [
    ['Answer concisely: 6+5=', '11'], # score 1
    ['Answer concisely: 19+87=', '106'], # score 2
    ['Answer concisely: 774+229=', '1003'], # score 3
    ['Answer concisely: 6587+3416=', '10003'], # score 4
    ['Answer concisely: 22605+77398=', '100003'], # score 5
    ['Answer concisely: 532847+467159=', '1000006'], # score 6
    ['Answer concisely: 5613709+4386294=', '10000003'], # score 7
    ['Answer concisely: 72582383+27417619=', '100000002'], # score 8
    ['Answer concisely: 206727644+793272359=', '1000000003'], # score 9
    ['Answer concisely: 7580116456+2419883549=', '10000000005'], # score 10
    ['Answer concisely: 52449010267+47550989737=', '100000000004'], # score 11
    ['Answer concisely: 888522030597+111477969406=', '1000000000003'], # score 12
    ['Answer concisely: 3764410205738+6235589794265=', '10000000000003'], # score 13
    ['Answer concisely: 45928371046529+54071628953474=', '100000000000003'], # score 14
    ['Answer concisely: 712850394621847+287149605378157=', '1000000000000004'], # score 15
]

# Cascading borrow tests
# Smaller number minus larger number, requires cascading borrows, results in negative answers
sub_tests = [
    ['Answer concisely: 23-47=', '-24'], # score 1
    ['Answer concisely: 156-289=', '-133'], # score 2
    ['Answer concisely: 2047-5183=', '-3136'], # score 3
    ['Answer concisely: 13056-27394=', '-14338'], # score 4
    ['Answer concisely: 240157-583269=', '-343112'], # score 5
    ['Answer concisely: 1350268-2794736=', '-1444468'], # score 6
    ['Answer concisely: 24601379-51836472=', '-27235093'], # score 7
    ['Answer concisely: 135702468-269485731=', '-133783263'], # score 8
    ['Answer concisely: 2461035789-5138274692=', '-2677238903'], # score 9
    ['Answer concisely: 13570246801-27394851639=', '-13824604838'], # score 10
    ['Answer concisely: 246103578924-518372469157=', '-272268890233'], # score 11
    ['Answer concisely: 1357024680135-2739485163728=', '-1382460483593'], # score 12
    ['Answer concisely: 24610357892461-51837246915839=', '-27226889023378'], # score 13
    ['Answer concisely: 135702468013570-273948516372946=', '-138246048359376'], # score 14
    ['Answer concisely: 2461035789246103-5183724691583727=', '-2722688902337624'], # score 15
]

In [22]:
import re 

def is_ground_truth_correct(answer, ground_truth):
    """
    Returns True if the ground_truth appears as the final number in the answer, ignoring whitespace and punctuation.
    Accepts answers like '13', '13.', '13**', 'The answer is 13', '**13**', 'random text **13** random text', 'boxed{13}'.
    """
    # Remove trailing whitespace and punctuation
    answer_clean = answer.strip().rstrip('.!**')
    # Find all numbers in the answer
    numbers = re.findall(r'\d+', answer_clean)

    return (ground_truth == answer or
            "**"+ground_truth+"**" in answer or
            "boxed{"+ground_truth+"}" in answer or
            ""+ground_truth+" " in answer or
            (numbers and numbers[-1] == ground_truth))

In [24]:
assert is_ground_truth_correct("-14338", "-14338")
assert not is_ground_truth_correct("-14339", "-14338")
assert is_ground_truth_correct("boxed{-14338}", "-14338")
assert is_ground_truth_correct("**-14338**", "-14338")
assert is_ground_truth_correct("blah blah-14338 blah blah", "-14338")

## Martian AI Model Names List

Extracted from https://app.withmartian.com/docs/index.html
Includes model names, costs, and parameter counts where available

Model data with costs (per 1M tokens) and parameters
Format: (model_name, input_cost_per_1M, output_cost_per_1M, request_cost, estimated_params)

In [3]:

martian_models_data = [
    ("martian/code", 2.40, 12.00, 0.00, None),
    #("agentica-org/deepcoder-14b-preview", 0.02, 0.02, 0.00, "14B"),
    #("ai21/jamba-1.6-large", 2.00, 8.00, 0.00, None),
    #("ai21/jamba-1.6-mini", 0.20, 0.40, 0.00, None),
    ("aion-labs/aion-rp-llama-3.1-8b", 0.20, 0.20, 0.00, "8B"),
    ("alfredpros/codellama-7b-instruct-solidity", 0.80, 1.20, 0.00, "7B"),
    ("alpindale/goliath-120b", 9.00, 11.00, 0.00, "120B"),
    ("amazon/nova-lite-v1", 0.06, 0.24, 0.00, None),
    ("amazon/nova-micro-v1", 0.04, 0.14, 0.00, None),
    ("amazon/nova-pro-v1", 0.80, 3.20, 0.00, None),
    ("anthracite-org/magnum-v4-72b", 2.50, 3.00, 0.00, "72B"),
    ("anthropic/claude-3-5-haiku-20241022", 0.80, 4.00, 0.00, None),
    ("anthropic/claude-3-5-sonnet-20240620", 3.00, 15.00, 0.00, None),
    ("anthropic/claude-3-5-sonnet-20241022", 3.00, 15.00, 0.00, None),
    ("anthropic/claude-3-7-sonnet-20250219", 3.00, 15.00, 0.00, None),
    ("anthropic/claude-3-haiku-20240307", 0.25, 1.25, 0.00, None),
    ("anthropic/claude-3-opus-20240229", 15.00, 75.00, 0.00, None),
    ("anthropic/claude-opus-4-1-20250805", 15.00, 75.00, 0.00, None),
    ("anthropic/claude-opus-4-20250514", 15.00, 75.00, 0.00, None),
    ("anthropic/claude-sonnet-4-20250514", 3.00, 15.00, 0.00, None),
    #("arcee-ai/arcee-blitz", 0.45, 0.75, 0.00, None),
    #("arcee-ai/caller-large", 0.55, 0.85, 0.00, None),
    ("arcee-ai/coder-large", 0.50, 0.80, 0.00, None),
    ("arcee-ai/maestro-reasoning", 0.90, 3.30, 0.00, None),
    ("arcee-ai/virtuoso-large", 0.75, 1.20, 0.00, None),
    #("arliai/qwq-32b-arliai-rpr-v1", 0.02, 0.02, 0.00, "32B"),
    ("baidu/ernie-4.5-300b-a47b", 0.28, 1.10, 0.00, "300B"),
    ("bytedance/ui-tars-1.5-7b", 0.10, 0.20, 0.00, "7B"),
    #("cognitivecomputations/dolphin-mixtral-8x22b", 0.90, 0.90, 0.00, "8x22B"),
    #("cognitivecomputations/dolphin3.0-r1-mistral-24b", 0.01, 0.01, 0.00, "24B"),
    ("cohere/command", 1.00, 2.00, 0.00, None),
    ("cohere/command-a", 2.50, 10.00, 0.00, None),
    ("cohere/command-r", 0.50, 1.50, 0.00, None),
    ("cohere/command-r-03-2024", 0.50, 1.50, 0.00, None),
    ("cohere/command-r-08-2024", 0.15, 0.60, 0.00, None),
    ("cohere/command-r-plus", 3.00, 15.00, 0.00, None),
    ("cohere/command-r-plus-04-2024", 3.00, 15.00, 0.00, None),
    ("cohere/command-r-plus-08-2024", 2.50, 10.00, 0.00, None),
    ("cohere/command-r7b-12-2024", 0.04, 0.15, 0.00, "7B"),
    ("deepseek/deepseek-chat", 0.27, 0.27, 0.00, None),
    ("deepseek/deepseek-chat-v3-0324", 0.25, 0.85, 0.00, None),
    ("deepseek/deepseek-prover-v2", 0.50, 2.18, 0.00, None),
    ("deepseek/deepseek-r1", 0.40, 2.00, 0.00, None),
    ("deepseek/deepseek-r1-0528", 0.27, 0.27, 0.00, None),
    ("deepseek/deepseek-r1-0528-qwen3-8b", 0.01, 0.02, 0.00, "8B"),
    ("deepseek/deepseek-r1-distill-llama-70b", 0.05, 0.05, 0.00, "70B"),
    ("deepseek/deepseek-r1-distill-llama-8b", 0.04, 0.04, 0.00, "8B"),
    #("deepseek/deepseek-r1-distill-qwen-1.5b", 0.18, 0.18, 0.00, "1.5B"),
    ("deepseek/deepseek-r1-distill-qwen-14b", 0.15, 0.15, 0.00, "14B"),
    ("deepseek/deepseek-r1-distill-qwen-32b", 0.08, 0.15, 0.00, "32B"),
    #("deepseek/deepseek-r1-distill-qwen-7b", 0.10, 0.20, 0.00, "7B"),
    ("eleutherai/llemma_7b", 0.80, 1.20, 0.00, "7B"),
    #("eva-unit-01/eva-qwen-2.5-72b", 4.00, 6.00, 0.00, "72B"),
    ("google/gemini-2.0-flash-001", 0.10, 0.40, 0.00, None),
    ("google/gemini-2.0-flash-lite-001", 0.08, 0.30, 0.00, None),
    #("google/gemini-2.5-flash", 0.30, 2.50, 0.00, None),
    ("google/gemini-2.5-flash-lite", 0.10, 0.40, 0.00, None),
    ("google/gemini-2.5-flash-lite-preview-06-17", 0.10, 0.40, 0.00, None),
    ("google/gemini-2.5-pro", 1.25, 10.00, 0.00, None),
    ("google/gemini-2.5-pro-preview-05-06", 1.25, 10.00, 0.00, None),
    ("google/gemma-3-12b-it", 0.03, 0.03, 0.00, "12B"),
    ("google/gemma-3-27b-it", 0.09, 0.17, 0.00, "27B"),
    ("google/gemma-3-4b-it", 0.02, 0.04, 0.00, "4B"),
    ("google/gemma-3n-e4b-it", 0.02, 0.04, 0.00, "4B"),
    ("gryphe/mythomax-l2-13b", 0.06, 0.06, 0.00, "13B"),
    ("inception/mercury", 0.25, 1.00, 0.00, None),
    ("inception/mercury-coder", 0.25, 1.00, 0.00, None),
    #("infermatic/mn-inferor-12b", 0.80, 1.20, 0.00, "12B"),
    ("liquid/lfm-3b", 0.02, 0.02, 0.00, "3B"),
    #("liquid/lfm-40b", 0.15, 0.15, 0.00, "40B"),
    ("liquid/lfm-7b", 0.01, 0.01, 0.00, "7B"),
    ("mancer/weaver", 1.50, 1.50, 0.00, None),
    ("meta-llama/llama-3-70b-instruct", 0.30, 0.40, 0.00, "70B"),
    ("meta-llama/llama-3-8b-instruct", 0.03, 0.06, 0.00, "8B"),
    ("meta-llama/llama-3.1-405b", 2.00, 2.00, 0.00, "405B"),
    ("meta-llama/llama-3.1-70b-instruct", 0.10, 0.28, 0.00, "70B"),
    ("meta-llama/llama-3.1-8b-instruct", 0.02, 0.02, 0.00, "8B"),
    ("meta-llama/llama-3.2-11b-vision-instruct", 0.05, 0.05, 0.00, "11B"),
    ("meta-llama/llama-3.2-1b-instruct", 0.01, 0.01, 0.00, "1B"),
    ("meta-llama/llama-3.2-3b-instruct", 0.00, 0.01, 0.00, "3B"),
    ("meta-llama/llama-3.2-90b-vision-instruct", 1.20, 1.20, 0.00, "90B"),
    ("meta-llama/llama-3.3-70b-instruct", 0.04, 0.12, 0.00, "70B"),
    ("meta-llama/llama-4-maverick", 0.15, 0.60, 0.00, None),
    ("meta-llama/llama-4-scout", 0.08, 0.30, 0.00, None),
    ("meta-llama/llama-guard-2-8b", 0.20, 0.20, 0.00, "8B"),
    ("meta-llama/llama-guard-3-8b", 0.02, 0.06, 0.00, "8B"),
    ("meta-llama/llama-guard-4-12b", 0.05, 0.05, 0.00, "12B"),
    ("microsoft/phi-3-medium-128k-instruct", 1.00, 1.00, 0.00, None),
    ("microsoft/phi-3-mini-128k-instruct", 0.10, 0.10, 0.00, None),
    ("microsoft/phi-3.5-mini-128k-instruct", 0.10, 0.10, 0.00, None),
    ("microsoft/phi-4", 0.06, 0.14, 0.00, None),
    ("microsoft/phi-4-multimodal-instruct", 0.05, 0.10, 0.00, None),
    ("microsoft/phi-4-reasoning-plus", 0.07, 0.35, 0.00, None),
    ("microsoft/wizardlm-2-8x22b", 0.48, 0.48, 0.00, "8x22B"),
    ("minimax/minimax-m1", 0.30, 1.65, 0.00, None),
    ("mistralai/codestral-2501", 0.30, 0.90, 0.00, None),
    ("mistralai/devstral-medium", 0.40, 2.00, 0.00, None),
    ("mistralai/devstral-small", 0.07, 0.28, 0.00, None),
    ("mistralai/devstral-small-2505", 0.03, 0.03, 0.00, None),
    ("mistralai/magistral-medium-2506", 2.00, 5.00, 0.00, None),
    ("mistralai/magistral-medium-2506:thinking", 2.00, 5.00, 0.00, None),
    ("mistralai/magistral-small-2506", 0.50, 1.50, 0.00, None),
    ("mistralai/ministral-3b", 0.04, 0.04, 0.00, "3B"),
    ("mistralai/ministral-8b", 0.10, 0.10, 0.00, "8B"),
    ("mistralai/mistral-7b-instruct", 0.03, 0.05, 0.00, "7B"),
    ("mistralai/mistral-7b-instruct-v0.1", 0.11, 0.19, 0.00, "7B"),
    #("mistralai/mistral-7b-instruct-v0.2", 0.20, 0.20, 0.00, "7B"),
    ("mistralai/mistral-7b-instruct-v0.3", 0.03, 0.05, 0.00, "7B"),
    ("mistralai/mistral-large", 2.00, 6.00, 0.00, None),
    ("mistralai/mistral-large-2407", 2.00, 6.00, 0.00, None),
    ("mistralai/mistral-large-2411", 2.00, 6.00, 0.00, None),
    ("mistralai/mistral-medium-3", 0.40, 2.00, 0.00, None),
    ("mistralai/mistral-nemo", 0.01, 0.05, 0.00, None),
    ("mistralai/mistral-saba", 0.20, 0.60, 0.00, None),
    ("mistralai/mistral-small", 0.20, 0.60, 0.00, None),
    ("mistralai/mistral-small-24b-instruct-2501", 0.03, 0.03, 0.00, "24B"),
    ("mistralai/mistral-small-3.1-24b-instruct", 0.03, 0.03, 0.00, "24B"),
    ("mistralai/mistral-small-3.2-24b-instruct", 0.05, 0.10, 0.00, "24B"),
    ("mistralai/mistral-tiny", 0.25, 0.25, 0.00, None),
    ("mistralai/mixtral-8x22b-instruct", 0.90, 0.90, 0.00, "8x22B"),
    ("mistralai/mixtral-8x7b-instruct", 0.08, 0.24, 0.00, "8x7B"),
    ("mistralai/pixtral-large-2411", 2.00, 6.00, 0.00, None),
    ("moonshotai/kimi-k2", 0.14, 2.49, 0.00, None),
    #("moonshotai/kimi-vl-a3b-thinking", 0.04, 0.04, 0.00, "3B"),
    #("morph/morph-v2", 1.20, 2.70, 0.00, None),
    ("morph/morph-v3-fast", 1.20, 2.70, 0.00, None),
    ("morph/morph-v3-large", 1.20, 2.70, 0.00, None),
    ("neversleep/llama-3-lumimaid-70b", 4.00, 6.00, 0.00, "70B"),
    ("neversleep/llama-3.1-lumimaid-8b", 0.18, 1.00, 0.00, "8B"),
    ("neversleep/noromaid-20b", 1.25, 2.00, 0.00, "20B"),
    #("nothingiisreal/mn-celeste-12b", 0.80, 1.20, 0.00, "12B"),
    #("nousresearch/deephermes-3-mistral-24b-preview", 0.14, 0.14, 0.00, "24B"),
    ("nousresearch/hermes-2-pro-llama-3-8b", 0.03, 0.04, 0.00, "8B"),
    ("nousresearch/hermes-3-llama-3.1-405b", 0.70, 0.80, 0.00, "405B"),
    ("nousresearch/hermes-3-llama-3.1-70b", 0.10, 0.28, 0.00, "70B"),
    #("nousresearch/nous-hermes-2-mixtral-8x7b-dpo", 0.60, 0.60, 0.00, "8x7B"),
    ("nvidia/llama-3.1-nemotron-70b-instruct", 0.12, 0.30, 0.00, "70B"),
    ("nvidia/llama-3.1-nemotron-ultra-253b-v1", 0.60, 1.80, 0.00, "253B"),
    #("nvidia/llama-3.3-nemotron-super-49b-v1", 0.13, 0.40, 0.00, "49B"),
    ("openai/chatgpt-4o-latest", 5.00, 15.00, 0.00, None),
    ("openai/gpt-3.5-turbo", 0.50, 1.50, 0.00, None),
    ("openai/gpt-3.5-turbo-16k", 3.00, 4.00, 0.00, None),
    ("openai/gpt-4", 30.00, 60.00, 0.00, None),
    ("openai/gpt-4-0314", 30.00, 60.00, 0.00, None),
    ("openai/gpt-4-1106-preview", 10.00, 30.00, 0.00, None),
    ("openai/gpt-4-turbo", 10.00, 30.00, 0.00, None),
    ("openai/gpt-4-turbo-preview", 10.00, 30.00, 0.00, None),
    ("openai/gpt-4.1", 2.00, 8.00, 0.00, None),
    ("openai/gpt-4.1-mini", 0.40, 1.60, 0.00, None),
    ("openai/gpt-4.1-nano", 0.10, 0.40, 0.00, None),
    ("openai/gpt-4o", 2.50, 10.00, 0.00, None),
    ("openai/gpt-4o-2024-05-13", 5.00, 15.00, 0.00, None),
    ("openai/gpt-4o-2024-08-06", 2.50, 10.00, 0.00, None),
    ("openai/gpt-4o-2024-11-20", 2.50, 10.00, 0.00, None),
    ("openai/gpt-4o-mini", 0.15, 0.60, 0.00, None),
    ("openai/gpt-4o-mini-2024-07-18", 0.15, 0.60, 0.00, None),
    ("openai/gpt-4o-mini-search-preview", 0.15, 0.60, 0.03, None),
    ("openai/gpt-4o-search-preview", 2.50, 10.00, 0.04, None),
    ("openai/gpt-5", 1.25, 10.00, 0.00, None),
    ("openai/gpt-5-mini", 0.25, 2.00, 0.00, None),
    ("openai/gpt-5-nano", 0.05, 0.40, 0.00, None),
    ("openai/o1", 15.00, 60.00, 0.00, None),
    ("openai/o1-mini", 1.10, 4.40, 0.00, None),
    ("openai/o1-mini-2024-09-12", 1.10, 4.40, 0.00, None),
    #("openai/o1-preview", 15.00, 60.00, 0.00, None),
    #("openai/o1-preview-2024-09-12", 15.00, 60.00, 0.00, None),
    ("openai/o3-mini", 1.10, 4.40, 0.00, None),
    ("openai/o4-mini", 1.10, 4.40, 0.00, None),
    #("opengvlab/internvl3-14b", 0.20, 0.40, 0.00, "14B"),
    #("perplexity/r1-1776", 2.00, 8.00, 0.00, None),
    ("perplexity/sonar", 1.00, 1.00, 0.01, None),
    ("perplexity/sonar-deep-research", 2.00, 8.00, 0.00, None),
    ("perplexity/sonar-pro", 3.00, 15.00, 0.00, None),
    ("perplexity/sonar-reasoning", 1.00, 5.00, 0.01, None),
    ("perplexity/sonar-reasoning-pro", 2.00, 8.00, 0.00, None),
    #("pygmalionai/mythalion-13b", 0.80, 1.20, 0.00, "13B"),
    #("qwen/qwen-2-72b-instruct", 0.90, 0.90, 0.00, "72B"),
    ("qwen/qwen-2.5-72b-instruct", 0.10, 0.10, 0.00, "72B"),
    ("qwen/qwen-2.5-7b-instruct", 0.04, 0.10, 0.00, "7B"),
    ("qwen/qwen-2.5-coder-32b-instruct", 0.06, 0.15, 0.00, "32B"),
    ("qwen/qwen-2.5-vl-7b-instruct", 0.20, 0.20, 0.00, "7B"),
    ("qwen/qwen-max", 1.60, 6.40, 0.00, None),
    ("qwen/qwen-plus", 0.40, 1.20, 0.00, None),
    ("qwen/qwen-turbo", 0.05, 0.20, 0.00, None),
    ("qwen/qwen-vl-max", 0.80, 3.20, 0.00, None),
    ("qwen/qwen2.5-vl-32b-instruct", 0.20, 0.60, 0.00, "32B"),
    ("qwen/qwen2.5-vl-72b-instruct", 0.25, 0.75, 0.00, "72B"),
    ("qwen/qwen3-14b", 0.06, 0.24, 0.00, "14B"),
    ("qwen/qwen3-235b-a22b", 0.13, 0.60, 0.00, "235B"),
    #("qwen/qwen3-235b-a22b-07-25", 0.12, 0.59, 0.00, "235B"),
    ("qwen/qwen3-30b-a3b", 0.08, 0.29, 0.00, "30B"),
    ("qwen/qwen3-32b", 0.03, 0.03, 0.00, "32B"),
    ("qwen/qwen3-8b", 0.04, 0.14, 0.00, "8B"),
    ("qwen/qwen3-coder", 0.30, 0.30, 0.00, None),
    ("qwen/qwq-32b", 0.08, 0.15, 0.00, "32B"),
    ("raifle/sorcererlm-8x22b", 4.50, 4.50, 0.00, "8x22B"),
    #("sao10k/fimbulvetr-11b-v2", 0.80, 1.20, 0.00, "11B"),
    ("sao10k/l3-euryale-70b", 1.48, 1.48, 0.00, "70B"),
    ("sao10k/l3-lunaris-8b", 0.02, 0.05, 0.00, "8B"),
    ("sao10k/l3.1-euryale-70b", 0.65, 0.75, 0.00, "70B"),
    ("sao10k/l3.3-euryale-70b", 0.65, 0.75, 0.00, "70B"),
    #("sarvamai/sarvam-m", 0.02, 0.02, 0.00, None),
    #("shisa-ai/shisa-v2-llama3.3-70b", 0.03, 0.03, 0.00, "70B"),
    #("sophosympatheia/midnight-rose-70b", 0.80, 0.80, 0.00, "70B"),
    ("switchpoint/router", 0.85, 3.40, 0.00, None),
    ("tencent/hunyuan-a13b-instruct", 0.03, 0.03, 0.00, "13B"),
    ("thedrummer/anubis-70b-v1.1", 0.50, 0.80, 0.00, "70B"),
    ("thedrummer/anubis-pro-105b-v1", 0.50, 1.00, 0.00, "105B"),
    ("thedrummer/rocinante-12b", 0.20, 0.50, 0.00, "12B"),
    ("thedrummer/skyfall-36b-v2", 0.02, 0.02, 0.00, "36B"),
    ("thedrummer/unslopnemo-12b", 0.40, 0.40, 0.00, "12B"),
    #("thedrummer/valkyrie-49b-v1", 0.65, 1.00, 0.00, "49B"),
    #("thudm/glm-4-32b", 0.24, 0.24, 0.00, "32B"),
    ("thudm/glm-4.1v-9b-thinking", 0.04, 0.14, 0.00, "9B"),
    #("thudm/glm-z1-32b", 0.03, 0.03, 0.00, "32B"),
    #("tngtech/deepseek-r1t2-chimera", 0.30, 0.30, 0.00, None),
    ("undi95/remm-slerp-l2-13b", 0.70, 1.00, 0.00, "13B"),
    #("undi95/toppy-m-7b", 0.80, 1.20, 0.00, "7B"),
    ("x-ai/grok-2-1212", 2.00, 10.00, 0.00, None),
    ("x-ai/grok-2-vision-1212", 2.00, 10.00, 0.00, None),
    ("x-ai/grok-3", 3.00, 15.00, 0.00, None),
    ("x-ai/grok-3-beta", 3.00, 15.00, 0.00, None),
    ("x-ai/grok-3-mini", 0.30, 0.50, 0.00, None),
    ("x-ai/grok-3-mini-beta", 0.30, 0.50, 0.00, None),
    ("x-ai/grok-4", 3.00, 15.00, 0.00, None),
]

In [4]:
import re
from typing import List, Tuple, Optional, Dict, Any

In [5]:
# Helper functions for data analysis
def extract_model_names() -> List[str]:
    """Extract just the model names from the data"""
    return [model[0] for model in martian_models_data]

def extract_param_size(param_str: Optional[str]) -> Optional[float]:
    """Convert parameter string to float (in billions)"""
    if not param_str:
        return None
    
    # Remove 'B' and convert to float
    if param_str.endswith('B'):
        param_str = param_str[:-1]
    
    # Handle mixtral format like "8x22B" -> approximate total params
    if 'x' in param_str:
        parts = param_str.split('x')
        try:
            return float(parts[0]) * float(parts[1])
        except:
            return None
    
    try:
        return float(param_str)
    except:
        return None

def get_models_by_provider() -> Dict[str, List[Tuple[str, float, float, float, Optional[str]]]]:
    """Group models by provider"""
    providers = {}
    for model_data in martian_models_data:
        provider = model_data[0].split('/')[0]
        if provider not in providers:
            providers[provider] = []
        providers[provider].append(model_data)
    return providers

def get_cost_stats() -> Dict[str, Any]:
    """Get cost statistics"""
    input_costs = [model[1] for model in martian_models_data]
    output_costs = [model[2] for model in martian_models_data]
    
    return {
        'input_cost_range': (min(input_costs), max(input_costs)),
        'output_cost_range': (min(output_costs), max(output_costs)),
        'avg_input_cost': sum(input_costs) / len(input_costs),
        'avg_output_cost': sum(output_costs) / len(output_costs)
    }

def get_param_stats() -> Dict[str, Any]:
    """Get parameter count statistics"""
    params_with_data = [extract_param_size(model[4]) for model in martian_models_data if model[4]]
    params_with_data = [p for p in params_with_data if p is not None]
    
    if not params_with_data:
        return {'count': 0}
    
    return {
        'count': len(params_with_data),
        'range': (min(params_with_data), max(params_with_data)),
        'avg_params': sum(params_with_data) / len(params_with_data),
        'models_with_param_info': len([m for m in martian_models_data if m[4]]),
        'total_models': len(martian_models_data)
    }

def find_cheapest_models(top_n: int = 5) -> List[Tuple[str, float, float]]:
    """Find the cheapest models by input cost"""
    sorted_by_input = sorted(martian_models_data, key=lambda x: x[1])
    return [(model[0], model[1], model[2]) for model in sorted_by_input[:top_n]]

def find_most_expensive_models(top_n: int = 5) -> List[Tuple[str, float, float]]:
    """Find the most expensive models by input cost"""
    sorted_by_input = sorted(martian_models_data, key=lambda x: x[1], reverse=True)
    return [(model[0], model[1], model[2]) for model in sorted_by_input[:top_n]]

def find_largest_models(top_n: int = 10) -> List[Tuple[str, Optional[str], float]]:
    """Find the largest models by parameter count"""
    models_with_params = [(model[0], model[4], extract_param_size(model[4])) 
                         for model in martian_models_data if model[4]]
    models_with_params = [(m[0], m[1], m[2]) for m in models_with_params if m[2] is not None]
    sorted_by_params = sorted(models_with_params, key=lambda x: x[2], reverse=True)
    return sorted_by_params[:top_n]

In [6]:
martian_models = extract_model_names()

# Print comprehensive statistics

print("=== MARTIAN AI MODELS ANALYSIS ===\n")

print(f"📊 Total models: {len(martian_models_data)}")

# Provider breakdown
providers = get_models_by_provider()
print(f"🏢 Number of providers: {len(providers)}")
#print("\n📈 Models per provider:")
#for provider, models in sorted(providers.items(), key=lambda x: len(x[1]), reverse=True):
#    print(f"  {provider:20} {len(models):3d} models")

# Cost analysis
cost_stats = get_cost_stats()
print(f"\n💰 Cost Analysis (per 1M tokens):")
print(f"  Input cost range:  ${cost_stats['input_cost_range'][0]:.2f} - ${cost_stats['input_cost_range'][1]:.2f}")
print(f"  Output cost range: ${cost_stats['output_cost_range'][0]:.2f} - ${cost_stats['output_cost_range'][1]:.2f}")
print(f"  Average input:     ${cost_stats['avg_input_cost']:.2f}")
print(f"  Average output:    ${cost_stats['avg_output_cost']:.2f}")

# Parameter analysis
param_stats = get_param_stats()
print(f"\n🧠 Parameter Analysis:")
print(f"  Models with param info: {param_stats['models_with_param_info']}/{param_stats['total_models']}")
if param_stats['count'] > 0:
    print(f"  Parameter range: {param_stats['range'][0]:.1f}B - {param_stats['range'][1]:.1f}B")
    print(f"  Average size: {param_stats['avg_params']:.1f}B parameters")

# Top lists
print(f"\n💸 5 Cheapest Models (input cost):")
for i, (model, input_cost, output_cost) in enumerate(find_cheapest_models(), 1):
    print(f"  {i}. {model:40} ${input_cost:.2f}/${output_cost:.2f}")

print(f"\n💎 5 Most Expensive Models (input cost):")
for i, (model, input_cost, output_cost) in enumerate(find_most_expensive_models(), 1):
    print(f"  {i}. {model:40} ${input_cost:.2f}/${output_cost:.2f}")

print(f"\n🦣 10 Largest Models:")
for i, (model, param_str, param_float) in enumerate(find_largest_models(), 1):
    print(f"  {i:2d}. {model:40} {param_str:>8} ({param_float:.1f}B)")

print(f"\n📋 Sample model data structure:")
print(f"   Format: (model_name, input_cost_per_1M, output_cost_per_1M, request_cost, params)")
for i in range(3):
    model = martian_models_data[i]
    print(f"   {model}")

print(f"\n✅ Use 'martian_models' for just the model names list")
print(f"✅ Use 'martian_models_data' for full data with costs and parameters")

=== MARTIAN AI MODELS ANALYSIS ===

📊 Total models: 188
🏢 Number of providers: 38

💰 Cost Analysis (per 1M tokens):
  Input cost range:  $0.00 - $30.00
  Output cost range: $0.01 - $75.00
  Average input:     $1.62
  Average output:    $5.37

🧠 Parameter Analysis:
  Models with param info: 77/188
  Parameter range: 1.0B - 405.0B
  Average size: 53.9B parameters

💸 5 Cheapest Models (input cost):
  1. meta-llama/llama-3.2-3b-instruct         $0.00/$0.01
  2. deepseek/deepseek-r1-0528-qwen3-8b       $0.01/$0.02
  3. liquid/lfm-7b                            $0.01/$0.01
  4. meta-llama/llama-3.2-1b-instruct         $0.01/$0.01
  5. mistralai/mistral-nemo                   $0.01/$0.05

💎 5 Most Expensive Models (input cost):
  1. openai/gpt-4                             $30.00/$60.00
  2. openai/gpt-4-0314                        $30.00/$60.00
  3. anthropic/claude-3-opus-20240229         $15.00/$75.00
  4. anthropic/claude-opus-4-1-20250805       $15.00/$75.00
  5. anthropic/claude-opus-4-2

## Run Models

In [7]:
import os
from dotenv import load_dotenv
import openai
import concurrent.futures
import time
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [8]:
load_dotenv()
MARTIAN_API_KEY = os.getenv("MARTIAN_API_KEY")
assert MARTIAN_API_KEY, "API key not found. Please set MARTIAN_API_KEY in your .env file."

In [9]:
client = openai.OpenAI(
    base_url="https://api.withmartian.com/v1",
    api_key=MARTIAN_API_KEY
)

In [10]:
def run_model_inference(model_name, prompt, ground_truth, timeout=300):

    response = client.chat.completions.create(
        model=model_name,
        max_tokens=1024, # Mandatory param for some models. Ignored by others.
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content.strip()
    success = is_ground_truth_correct(answer, ground_truth)
    return answer, success

In [11]:
def evaluate_models_progressive(the_models, tests, max_workers=8):
    model_scores = []

    def score_model(model_name):
        score = 0
        for test_idx, (prompt, ground_truth) in enumerate(tests):
            try:
                answer, success = run_model_inference(model_name, prompt, ground_truth)
                if success:
                    score = test_idx + 1
                else:
                    break
            except openai.APIError as e:
                if hasattr(e, 'status_code'):
                    score = -e.status_code
                else:
                    score = -999
                break
            except Exception as e:
                score = -999
                break
        return {"model": model_name, "score": score}

    print(f"Evaluating {len(the_models)} models concurrently with {max_workers} workers...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_model = {executor.submit(score_model, model_name): model_name for model_name in the_models}
        for idx, future in enumerate(as_completed(future_to_model), 1):
            model_name = future_to_model[future]
            try:
                result = future.result()
            except Exception as exc:
                result = {"model": model_name, "score": -999}
            print(f"[{idx}/{len(the_models)}] {result['model']}: Score = {result['score']}")
            model_scores.append(result)
    return model_scores

In [12]:
# Use the updated martian_models_data structure to extract model names
the_models = [model[0] for model in martian_models_data]
add_model_scores = evaluate_models_progressive(the_models, add_tests)
sub_model_scores = evaluate_models_progressive(the_models, sub_tests)

Evaluating 188 models concurrently with 8 workers...
[1/188] alfredpros/codellama-7b-instruct-solidity: Score = 0
[1/188] alfredpros/codellama-7b-instruct-solidity: Score = 0
[2/188] amazon/nova-lite-v1: Score = 2
[2/188] amazon/nova-lite-v1: Score = 2
[3/188] amazon/nova-micro-v1: Score = 3
[3/188] amazon/nova-micro-v1: Score = 3
[4/188] anthropic/claude-3-5-haiku-20241022: Score = 2
[4/188] anthropic/claude-3-5-haiku-20241022: Score = 2
[5/188] martian/code: Score = 2
[6/188] alpindale/goliath-120b: Score = 3
[5/188] martian/code: Score = 2
[6/188] alpindale/goliath-120b: Score = 3
[7/188] aion-labs/aion-rp-llama-3.1-8b: Score = 3
[8/188] anthropic/claude-3-5-sonnet-20240620: Score = 3
[7/188] aion-labs/aion-rp-llama-3.1-8b: Score = 3
[8/188] anthropic/claude-3-5-sonnet-20240620: Score = 3
[9/188] anthropic/claude-3-opus-20240229: Score = 2
[9/188] anthropic/claude-3-opus-20240229: Score = 2
[10/188] anthropic/claude-3-7-sonnet-20250219: Score = 3
[10/188] anthropic/claude-3-7-sonnet

# Summarize results

In [13]:
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
 
def plot_model_scores(model_scores, title):
    # Summarize number of models by score
    print(title)
    print(f"Total number of models: {len(martian_models)}")
    score_counts = Counter([m['score'] for m in model_scores])
    print("Score summary (number of models by score):")
    for score, count in sorted(score_counts.items()):
        print(f"Score {score}: {count} models") 

    # Print total number of models with negative and non-negative scores
    num_negative = sum(count for score, count in score_counts.items() if isinstance(score, int) and score < 0)
    num_nonneg = sum(count for score, count in score_counts.items() if isinstance(score, int) and score >= 0)
    print(f"\nTotal models with negative score: {num_negative}")
    print(f"Total models with non-negative score: {num_nonneg}")

    # Print percentage of models with non-negative scores that scored 10 or more
    num_10_or_more = sum(count for score, count in score_counts.items() if isinstance(score, int) and score >= 10)
    if num_nonneg > 0:
        percent_10_or_more = 100 * num_10_or_more / num_nonneg
        print(f"Percentage of non-negative models scoring 10 or more: {percent_10_or_more:.2f}%")
    else:
        print("No models with non-negative scores.")
    print("\n")

    return score_counts

add_score_counts = plot_model_scores(add_model_scores, "Addition Test Scores")
sub_score_counts = plot_model_scores(sub_model_scores, "Subtraction Test Scores")

Addition Test Scores
Total number of models: 188
Score summary (number of models by score):
Score -502: 6 models
Score 0: 8 models
Score 1: 6 models
Score 2: 25 models
Score 3: 43 models
Score 4: 37 models
Score 5: 35 models
Score 6: 8 models
Score 7: 2 models
Score 8: 4 models
Score 9: 4 models
Score 10: 2 models
Score 11: 1 models
Score 12: 3 models
Score 15: 4 models

Total models with negative score: 6
Total models with non-negative score: 182
Percentage of non-negative models scoring 10 or more: 5.49%


Subtraction Test Scores
Total number of models: 188
Score summary (number of models by score):
Score -502: 6 models
Score 0: 168 models
Score 1: 7 models
Score 2: 3 models
Score 3: 4 models

Total models with negative score: 6
Total models with non-negative score: 182
Percentage of non-negative models scoring 10 or more: 0.00%




In [14]:
# Bar graph for Addition and Subtraction score counts, x-axis always 0 to 15
score_range = list(range(16))
add_counts = [add_score_counts.get(s, 0) for s in score_range]
sub_counts = [sub_score_counts.get(s, 0) for s in score_range]

plt.figure(figsize=(12, 5))
bars_add = plt.bar([x-0.2 for x in score_range], add_counts, width=0.4, color='skyblue', label='Addition')
bars_sub = plt.bar([x+0.2 for x in score_range], sub_counts, width=0.4, color='salmon', label='Subtraction')

plt.xlabel('Model Score', fontsize=18)
plt.ylabel('Number of Models', fontsize=18)
plt.xticks(score_range, [str(s) for s in score_range], fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=16)

# Add count labels inside bars only where count > 4, with larger font
for bar, count in zip(bars_add, add_counts):
    if count > 4:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height()/2, str(count), ha='center', va='center', fontsize=18, color='black')
for bar, count in zip(bars_sub, sub_counts):
    if count > 4:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height()/2, str(count), ha='center', va='center', fontsize=18, color='black')

plt.tight_layout()
plt.show()

In [22]:
# Scatter plot: Output Token Cost vs. Model Score for Addition and Subtraction (x-axis always 0 to 15)
import numpy as np
model_name_to_output_cost = {model[0]: model[2] for model in martian_models_data}

# Addition scores
add_model_names = [entry['model'] for entry in add_model_scores]
add_scores = [entry['score'] for entry in add_model_scores]
add_output_costs = [model_name_to_output_cost.get(name, np.nan) for name in add_model_names]
add_filtered = [(score, cost) for score, cost in zip(add_scores, add_output_costs) if isinstance(score, int) and 0 <= score <= 15]
if add_filtered:
    add_filtered_scores, add_filtered_output_costs = zip(*add_filtered)
else:
    add_filtered_scores, add_filtered_output_costs = [], []

# Subtraction scores
sub_model_names = [entry['model'] for entry in sub_model_scores]
sub_scores = [entry['score'] for entry in sub_model_scores]
sub_output_costs = [model_name_to_output_cost.get(name, np.nan) for name in sub_model_names]
sub_filtered = [(score, cost) for score, cost in zip(sub_scores, sub_output_costs) if isinstance(score, int) and 0 <= score <= 15]
if sub_filtered:
    sub_filtered_scores, sub_filtered_output_costs = zip(*sub_filtered)
else:
    sub_filtered_scores, sub_filtered_output_costs = [], []

# Prepare categorical x-axis for scores 0 to 15
score_range = list(range(16))
score_to_x = {score: i for i, score in enumerate(score_range)}
add_x_vals = [score_to_x[score] for score in add_filtered_scores]
sub_x_vals = [score_to_x[score] for score in sub_filtered_scores]

plt.figure(figsize=(10,6))
plt.scatter(add_x_vals, add_filtered_output_costs, alpha=0.6, color='blue', label='Addition')
plt.scatter(sub_x_vals, sub_filtered_output_costs, alpha=0.6, color='red', label='Subtraction')
plt.xlabel('Model Score', fontsize=18)
plt.ylabel('Model Output Token Cost (USD per 1M tokens)', fontsize=16)
plt.xticks(score_range, [str(s) for s in score_range], fontsize=16)
plt.yticks(fontsize=16)
plt.yscale('log')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(fontsize=16)
plt.tight_layout()
plt.show()

In [16]:
# List only models with a negative score, zero score or score >= 10 for both Addition and Subtraction
def list_models_by_score(model_scores, label):
    score_to_models = {}
    for entry in model_scores:
        score = entry['score']
        if score <= 0 or score >= 10:
            score_to_models.setdefault(score, []).append(entry['model'])
    print(f"\n{label}:")
    for score in sorted(score_to_models):
        print(f"Models with score {score}:")
        for model in score_to_models[score]:
            print(f"  {model}")

list_models_by_score(add_model_scores, "Addition Models")
list_models_by_score(sub_model_scores, "Subtraction Models")


Addition Models:
Models with score -502:
  cohere/command
  cohere/command-r
  cohere/command-r-03-2024
  cohere/command-r-plus
  cohere/command-r-plus-04-2024
  deepseek/deepseek-prover-v2
Models with score 0:
  alfredpros/codellama-7b-instruct-solidity
  eleutherai/llemma_7b
  mancer/weaver
  meta-llama/llama-guard-3-8b
  meta-llama/llama-guard-4-12b
  mistralai/mixtral-8x22b-instruct
  morph/morph-v3-fast
  meta-llama/llama-3.1-405b
Models with score 10:
  tencent/hunyuan-a13b-instruct
  baidu/ernie-4.5-300b-a47b
Models with score 11:
  mistralai/mistral-large-2407
Models with score 12:
  anthropic/claude-3-haiku-20240307
  amazon/nova-pro-v1
  perplexity/sonar-deep-research
Models with score 15:
  arcee-ai/virtuoso-large
  openai/gpt-4o-mini-search-preview
  openai/gpt-4o-search-preview
  qwen/qwen3-coder

Subtraction Models:
Models with score -502:
  cohere/command-r
  cohere/command
  cohere/command-r-03-2024
  cohere/command-r-plus
  cohere/command-r-plus-04-2024
  meta-llama/l

# Check zero-score model output

On occasion, check the output of the "0 score" models to see if unusual answer formats are hiding correct answers.
If so, is_ground_truth_correct should be updated

In [17]:
# Re-run models that scored 0 on test[0] and show their outputs
def run_zero_score_models(title, model_scores, tests):
    zero_score_models = [m['model'] for m in model_scores if m['score'] == 0]
    print(f"Re-running {len(zero_score_models)} models that scored 0 on the first test for {title}...")
    outputs = []
    for model_name in zero_score_models:
        try:
            answer, success = run_model_inference(model_name, tests[0][0], tests[0][1])
        except Exception as e:
            answer = f"Error: {str(e)}"
            success = False
        outputs.append({'model': model_name, 'output': answer, 'success': success})
        print(f"Model: {model_name}\nOutput: {answer}\nSuccess: {success}\n{'-'*40}")


#run_zero_score_models("Addition", add_model_scores, add_tests)
#run_zero_score_models("Subtraction", sub_model_scores, sub_tests)

# Check -400 score models

Models can generate -400 scores if a mandatory parameter is missing. This may hide a model that can do addition.
Run the models that give a -400 score and see if the detailed error message shows the root cause.

In [18]:
# Re-run models that scored -400 and show their outputs and errors
def run_minus_400_score_models(title, model_scores, tests):
    minus_400_models = [m['model'] for m in model_scores if m['score'] == -400]
    print(f"Re-running {len(minus_400_models)} models that scored -400 on the first test for {title}...")
    outputs = []
    for model_name in minus_400_models:
        try:
            answer, success = run_model_inference(model_name, tests[0][0], tests[0][1])
        except Exception as e:
            answer = f"Error: {str(e)}"
            success = False
        outputs.append({'model': model_name, 'output': answer, 'success': success})
        print(f"Model: {model_name}\nOutput/Error: {answer}\nSuccess: {success}\n{'-'*40}")

#run_minus_400_score_models("Addition", add_model_scores, add_tests)
#run_minus_400_score_models("Subtraction", sub_model_scores, sub_tests)

# High Variability Example
The model "alfredpros/codellama-7b-instruct-solidity" has high variability. Sometimes answer is correct

In [19]:
def run_high_variability_model():
    model_name = "alfredpros/codellama-7b-instruct-solidity"
    for i in range(5):
        answer, success = run_model_inference(model_name, add_tests[0][0], add_tests[0][1])
        print(f"Success: {success}. Output: {answer}")

#run_high_variability_model()

# Ask how the models that score >=14 work

Inconclusive: 

Sometimes the detailed methodology is mathematically incorrect.
Sometimes the outline methodology looks like a sentence from a text book.
The bext valid explanations say the model does addition like a human - line up digit pairs - then sum right digit-pair to-left digit pair calculating carries 
The "concise" setting makes these model reply immediately - unless they have many hidden "thinking tokens" the human approach is not feasible.

Some of these models gpt-4o-mini-search-preview and gpt-4o-search-preview explicitly say they have search/tool access. 
So they can just call a python function to calculate the sum.

In [20]:
def show_work_for_score_best_models(title, model_scores):
    best_models = [m['model'] for m in model_scores if m['score'] >= 14]
    ground_prompt = "998522030597+1477969406="
    concise_prompt = f"Answer concisely: {ground_prompt}"
    explain_prompt = f"Show your work for {ground_prompt}"
    ground_truth = str(998522030597 + 1477969406)

    print(f"Querying {len(best_models)} models that scored best...")
    for model_name in best_models:
        try:
            answer, _ = run_model_inference(model_name, concise_prompt, ground_truth)
        except Exception as e:
            answer = f"Error: {str(e)}"
        print(f"Model: {model_name}\nOutput: {answer}\n{'-'*40}")

        try:
            answer, _ = run_model_inference(model_name, explain_prompt, ground_truth)
        except Exception as e:
            answer = f"Error: {str(e)}"
        print(f"Model: {model_name}\nOutput: {answer}\n{'-'*40}")

#show_work_for_score_best_models("Addition", add_model_scores)
#show_work_for_score_best_models("Subtraction", sub_model_scores)

# Chat GPT 5
What did the Chat GPT 5 models score?

In [21]:
# Print scores for all models containing 'gpt-5' in both Addition and Subtraction
def print_gpt5_scores(model_scores, label):
    print(f"\n{label}:")
    for entry in model_scores:
        model = entry['model']
        score = entry['score']
        if 'gpt-5' in model:
            print(f"Model: {model}, Score: {score}")

print_gpt5_scores(add_model_scores, "Addition Scores")
print_gpt5_scores(sub_model_scores, "Subtraction Scores")


Addition Scores:
Model: openai/gpt-5-mini, Score: 5
Model: openai/gpt-5, Score: 6
Model: openai/gpt-5-nano, Score: 6

Subtraction Scores:
Model: openai/gpt-5-mini, Score: 0
Model: openai/gpt-5, Score: 0
Model: openai/gpt-5-nano, Score: 0


# Test some subtraction examples 

In [30]:
import random
from pprint import pprint

# Filter models that scored more than 0 on subtraction
eligible_models = [entry for entry in sub_model_scores if entry['score'] > 0]

if not eligible_models:
    print("No models scored more than 0 on subtraction.")
else:
    # Pick up to 5 random models
    selected_models = random.sample(eligible_models, min(5, len(eligible_models)))

    for entry in selected_models:
        model = entry['model']
        score = entry['score']
        print(f"\nModel: {model} (Subtraction Score: {score})")
        # Run at the score they succeeded at
        if 1 <= score <= 15:
            test_idx = score - 1
            test_prompt, test_answer = sub_tests[test_idx]
            print(f"Running at succeeded test (Score {score}): {test_prompt}")
            answer1, success1 = run_model_inference(model, test_prompt, test_answer)
            print(test_answer, success1, answer1)
            # Run at the next harder test they failed at
            if score < 15:
                next_test_idx = score
                next_test_prompt, next_test_answer = sub_tests[next_test_idx]
                print(f"Running at next harder test (Score {score+1}): {next_test_prompt}")
                answer2, success2 = run_model_inference(model, next_test_prompt, next_test_answer)
                print(next_test_answer, success2, answer2)
        else:
            print("Model did not succeed at any subtraction test in the defined range.")


Model: deepseek/deepseek-r1 (Subtraction Score: 2)
Running at succeeded test (Score 2): Answer concisely: 156-289=
-133 True The result of 156 − 289 is **-133**.  
*Explanation:* Since 289 > 156, subtract 156 from 289 to get 133, then apply the negative sign for -133.
Running at next harder test (Score 3): Answer concisely: 2047-5183=
-3136 None -3136

Model: deepseek/deepseek-r1-distill-qwen-14b (Subtraction Score: 3)
Running at succeeded test (Score 3): Answer concisely: 2047-5183=
-3136 <re.Match object; span=(327, 339), match='boxed{-3136}'> To solve the subtraction problem \( 2047 - 5183 \) concisely, follow these steps:

1. **Line Up the Numbers:**

   ```
     2047
   - 5183
   ```

2. **Perform the Subtraction:**

   Since \( 2047 \) is smaller than \( 5183 \), the result will be negative. 

   \[
   2047 - 5183 = -3136
   \]

3. **Final Answer:**

   \[
   \boxed{-3136}
   \]
Running at next harder test (Score 4): Answer concisely: 13056-27394=


KeyboardInterrupt: 